#### Test mynds packages in the sandbox

In [2]:
from pathlib import Path
from typing import Any

import Metashape as ms
import polars as pl

import mynd.backend.metashape as backend

from mynd.io import read_data_frame
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def work_with_project(metadata: pl.DataFrame) -> None:
    """TODO"""

    # NOTE: Write your code here
    document: ms.Document = backend.context._backend_data.get("document")
    chunk: ms.Chunk = document.chunks[0]

    for camera in chunk.cameras[:10]:

        # NOTE: This is how you write metadata to cameras
        # NOTE: Metashape only allows metadata to be store as strings
        camera.meta["greeting"] = str(10)

        if not camera.meta:
            continue
        else:
            logger.info(camera.meta)

            # NOTE: This is how you retrieve metadata from cameras
            items: list[tuple[Any, Any]] = camera.meta.items()
            keys: list[Any] = camera.meta.keys()
            values: list[Any] = camera.meta.values()

    save: bool = False
    if save:
        match backend.save_project(DESTINATION_PROJECT):
            case Ok(path):
                logger.info(f"successfully saved project to: {path}")
            case Err(message):
                logger.error(f"failed to save project: {message}")


def main() -> None:
    """Main entrypoint."""

    PATHS: dict[str, Path] = {
        "source": Path(
            "/data/kingston_snv_01/acfr_metashape_projects_dev/r23685bc_lite.psz"
        ),
        "destination": Path(
            "/data/kingston_snv_01/acfr_metashape_projects_dev/r23685bc_lite_metadata.psz"
        ),
        "metadata": Path(
            "/data/kingston_snv_01/acfr_cameras_metadata/r23685bc_20100605_021022_camera_metadata.csv"
        ),
    }

    # Options
    save: bool = False

    match read_data_frame(PATHS.get("metadata")):
        case Ok(metadata):
            pass
        case Err(message):
            logger.error(message)
            return

    match backend.load_project(PATHS.get("source")):
        case Ok(path):
            work_with_project(metadata)
        case Err(message):
            logger.error(message)
            return

    if save:
        match backend.save_project(PATHS.get("destination")):
            case Ok(path):
                logger.info(f"successfully saved project to: {path}")
            case Err(message):
                logger.error(f"failed to save project: {message}")


# INVOKE MAIN
main()

2024-10-05 14:26:30.669 | ERROR    | __main__:main:74 - backend already has a loaded project
